## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-28-16-50-32 +0000,bbc,Zelensky's chief of staff resigns after Ukrain...,https://www.bbc.com/news/articles/ckg9nd2wddno...
1,2025-11-28-16-48-47 +0000,wapo,Death toll in Hong Kong high-rise fire grows t...,https://www.washingtonpost.com/world/2025/11/2...
2,2025-11-28-16-48-13 +0000,nyt,Officials Had Been Warned for Over a Year Befo...,https://www.nytimes.com/2025/11/28/world/asia/...
3,2025-11-28-16-46-00 +0000,wsj,Vivendi Takeover by Bollore Less Likely After ...,https://www.wsj.com/business/vivendi-takeover-...
4,2025-11-28-16-43-04 +0000,startribune,"Live: Shoppers out on Black Friday for fun, bu...",https://www.startribune.com/black-friday-shopp...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2335/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
164,trump,24
118,shooting,21
191,national,19
192,guard,17
119,suspect,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
195,2025-11-28-00-57-00 +0000,nypost,Trump claims National Guard shooting suspect ‘...,https://nypost.com/2025/11/27/us-news/trump-cl...,104
226,2025-11-27-20-14-05 +0000,nypost,Prosecutors will seek death penalty against Af...,https://nypost.com/2025/11/27/us-news/prosecut...,97
191,2025-11-28-01-08-59 +0000,wapo,"National Guard member dies after shooting, Tru...",https://www.washingtonpost.com/dc-md-va/2025/1...,95
26,2025-11-28-15-48-03 +0000,nyt,Trump Uses National Guard Shooting to Cast Sus...,https://www.nytimes.com/2025/11/27/us/politics...,86
80,2025-11-28-13-25-53 +0000,wapo,The National Guard shooting suspect was in one...,https://www.washingtonpost.com/nation/2025/11/...,84


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
195,104,2025-11-28-00-57-00 +0000,nypost,Trump claims National Guard shooting suspect ‘...,https://nypost.com/2025/11/27/us-news/trump-cl...
56,68,2025-11-28-14-45-24 +0000,nyt,Death Toll Rises to 128 in Hong Kong High-Rise...,https://www.nytimes.com/2025/11/28/world/asia/...
16,47,2025-11-28-16-08-10 +0000,nypost,Zelensky’s top aide Andriy Yermak resigns afte...,https://nypost.com/2025/11/28/world-news/zelen...
114,44,2025-11-28-11-00-00 +0000,nypost,"New analysis reveals where NYers, tourists are...",https://nypost.com/2025/11/28/us-news/new-anal...
22,39,2025-11-28-15-55-48 +0000,nypost,Haunting photos show LI teen Emily Finn and ex...,https://nypost.com/2025/11/28/us-news/haunting...
9,39,2025-11-28-16-23-34 +0000,nyt,Israeli Forces Kill Two Palestinians in West B...,https://www.nytimes.com/2025/11/28/world/middl...
226,39,2025-11-27-20-14-05 +0000,nypost,Prosecutors will seek death penalty against Af...,https://nypost.com/2025/11/27/us-news/prosecut...
167,32,2025-11-28-05-42-45 +0000,nypost,President Trump says he’ll ‘permanently pause ...,https://nypost.com/2025/11/28/us-news/presiden...
10,30,2025-11-28-16-22-26 +0000,bbc,No 10 denies Reeves misled public in run-up to...,https://www.bbc.com/news/articles/c4gex225p5jo...
108,28,2025-11-28-11-20-00 +0000,wsj,"Tech, Media & Telecom Roundup: Market Talk",https://www.wsj.com/business/tech-media-teleco...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
